# Airplane Data Visualization and Analysis

In [ ]:
import pandas as pd
import numpy as np
import holoviews as hv
import hvplot.pandas
import calendar
import panel as pn
import geoviews as gv 
import geopandas as gpd
from bokeh.models import HoverTool, Legend

## Cleaning Merged Passenger Datasets 

To analyze passenger data from both the most busy and least busy airports, we read the merged dataset `passengers_busy_airports.csv` and `passengers_less_busy_airports.csv` using `read_csv` function of pandas. To get more insight into our data, we use `info` and `head` functions so that we can process our datasets.

Reading the cleaned dataset on passengers from the most busy airports `passengers_busy_airports.csv` and the least busy airports `passengers_less_busy_airports.csv` using read_csv function of pandas

In [ ]:
psg_busy = pd.read_csv('data/passengers_busy_airports.csv')
psg_l_busy = pd.read_csv('data/passengers_less_busy_airports.csv')

In [ ]:
psg_busy.info()

In [ ]:
psg_busy

Now we remove rows with the label 'TOTAL' from the 'Month' column in both datasets. Additionally, as per the datatype for each columns of `psg_busy`, we are able to see that the numbers are separated by commas, so we converts numeric columns by using `map` function combined with a lambda function that removes commas and converts the values to integers, ensuring consistency for further processing and analysis. To improve data visualization, the 'Month' column in the most busy airports dataset is updated with abbreviated month names using `calendar.month_abbr`.

In [ ]:
busy_passengers = psg_busy[psg_busy.Month != 'TOTAL'].map(
    lambda x: int(x.replace(',','')) if type(x) == object and not x.isalpha() else x)

l_busy_passengers = psg_l_busy[psg_l_busy.Month != 'TOTAL'].map(
    lambda x: int(x.replace(',','')) if type(x) == object and not x.isalpha() else x)

busy_passengers['Month'] = busy_passengers['Month'].map(
    lambda x: calendar.month_abbr[int(x)])

columns_to_convert = ['DOMESTIC', 'INTERNATIONAL', 'TOTAL']

for column in columns_to_convert:
    busy_passengers[column] = busy_passengers[column].str.replace(',', '').astype(int)

For each of the airports, we scale down the data in order of 10 raised to power 4.

In [ ]:
psg_busy_airpt = []
busy_cities = ['Atlanta', 'Dallas', 'NewYorkCity', 'Seattle', 'Philadelphia']
less_busy_cities = ['Boise','Kansas','Oakland','Sacramento','Santa']

cities = {city : busy_passengers.City == city for city in busy_cities}

busy_passengers['DOMESTIC'] = busy_passengers['DOMESTIC'].map(lambda x: x /(10**4))
busy_passengers['INTERNATIONAL'] = busy_passengers['INTERNATIONAL'].apply(lambda x : x / (10**4))
busy_passengers['TOTAL'] = busy_passengers['TOTAL'].apply(lambda x : x / (10**4))

In [ ]:
busy_passengers

# Total Rows_Busy Airports

We are gathering the rows from the passenger and airfare dataset to create a separate dataframe for the yearly total

In [ ]:
p_b_total_rows = psg_busy[psg_busy['Month'] == 'TOTAL'].loc[:, psg_busy.columns != 'Month']
p_l_b_total_rows = psg_l_busy[psg_l_busy['Month'] == 'TOTAL'].loc[:, psg_l_busy.columns != 'Month']
airfare_l_busy = pd.read_csv('data/airfare_less_busy_airports.csv')
airfare_busy_yearly = airfare_l_busy.groupby(['Year', 'City']).sum().reset_index()
airfare_l_busy_yearly = airfare_l_busy.groupby(['Year', 'City']).sum().reset_index()
airfare_l_busy_yearly .head(25).loc[:, airfare_l_busy_yearly.columns != 'Quarter']

# Airfare data

The airfare data is further grouped based on year and City to gather the yearly data

In [ ]:
years_list = list(range(2005, 2023))
year_masks = {year: busy_passengers.Year == year for year in years_list}

busy_cities = ['Atlanta', 'Dallas', 'New York', 'Seattle', 'Philadelphia']
hv.extension('bokeh')
airfare_busy = pd.read_csv('data/airfare_busy_airports.csv')

grouped_airfare_busy = airfare_busy.groupby(by=['Year','City']).mean().reset_index()
grouped_airfare_busy.drop(columns='Quarter', inplace=True)
grouped_airfare_busy


In [ ]:
grp_airfare_busy_airpt= []
cities = {city : busy_passengers.City == city for city in busy_cities}
airfares = {city : grouped_airfare_busy.City == city for city in busy_cities}

In [ ]:
for city,mask in airfares.items():
    grp_airfare_busy_airpt.append(
        grouped_airfare_busy[airfares[city]].hvplot.line(x='Year',y='Current_City_Avg',label = f'{city}')
        * grouped_airfare_busy[airfares[city]].hvplot.scatter(x='Year',y='Current_City_Avg', label = f'{city}')
        )
hv.extension('bokeh')
(
grp_airfare_busy_airpt[0] * grp_airfare_busy_airpt[1] * grp_airfare_busy_airpt[2] * grp_airfare_busy_airpt[3]
* grp_airfare_busy_airpt[4]
).opts(
    title = 'Airfare Price changes',
    ylabel = 'Current Cuty Avg price (USD)',
    width = 900,
    height = 500
)

The plot above depicts the Airfare price variation in `USD` across the years `1995 to 2023`.

## International data Plotting across City

Now to get further insight on the passenger count across the airports with the airfare data, we group these data based on `Year` and `City`

In [ ]:
combine_psg_airfare = grouped_airfare_busy.merge(p_b_total_rows, left_on=('Year','City'), right_on=('Year','City'))

columns_to_convert = ['DOMESTIC', 'INTERNATIONAL', 'TOTAL']

for column in columns_to_convert:
    combine_psg_airfare[column] = combine_psg_airfare[column].str.replace(',', '').astype(int)

combine_psg_airfare['INTERNATIONAL'] = combine_psg_airfare['INTERNATIONAL'].apply(lambda x : x / (10**4))
combine_psg_airfare['DOMESTIC'] = combine_psg_airfare['DOMESTIC'].apply(lambda x : x / (10**4))
combine_psg_airfare['TOTAL'] = combine_psg_airfare['TOTAL'].apply(lambda x : x / (10**4))

In [ ]:
international_busy_cities = []

years_list = list(range(2003, 2023))

for city,mask in airfares.items():
   international_busy_cities.append(
      combine_psg_airfare[combine_psg_airfare.City == city].hvplot.line(y='Current_City_Avg',x='INTERNATIONAL',label = f'{city}').opts(tools = ['hover'])
      * combine_psg_airfare[combine_psg_airfare.City == city].hvplot.scatter(y='Current_City_Avg',x='INTERNATIONAL', label = f'{city}').opts(tools = ['hover'])
      )

(
   international_busy_cities[0] * international_busy_cities[1] * international_busy_cities[2] * international_busy_cities[3] * international_busy_cities[4]
).opts(
    title = 'Passengers count in busy airports and airfare across cities',
    xlabel = 'International Passengers ($10^{4}$)',
    ylabel = 'Current_City_Avg Airfare value in USD',
    tools = ['hover'],
    height=700,
    width = 900,
    show_grid=True,
    xlim = (20, 750)
)

### Domestic Data plot across cities - Busy Airports

Similar to the International passenger count, we also plot the Domestic data.

In [ ]:
domestic_busy_cities = []

for city,mask in airfares.items():
    domestic_busy_cities.append(
    combine_psg_airfare[combine_psg_airfare.City == city].hvplot.line(y='Current_City_Avg',x='DOMESTIC',label = f'{city}')
    * combine_psg_airfare[combine_psg_airfare.City == city].hvplot.scatter(y='Current_City_Avg',x='DOMESTIC', label = f'{city}')
    )

(
    domestic_busy_cities[0] *
   domestic_busy_cities[1] * domestic_busy_cities[2] * domestic_busy_cities[3] * domestic_busy_cities[4]
).opts(
    title = 'Domestic Passengers in busy airports and airfare from 2003 till 2023',
    xlabel = 'Domestic Passengers ($10^{4}$)',
    ylabel = 'Current_City_Avg Airfare value in USD',
    height=700,
    width = 1000,
    show_grid=True,
    xlim = (400, 5000)
)

## Less Busy Airport data with airfare

As we have plotted the busy airport data, we process to perform the same for less busy airport.

In [ ]:
airfare_l_busy_yearly = airfare_l_busy.groupby(['Year', 'City']).mean().reset_index()

# Total passangers per year across cities
p_l_b_total_rows = psg_l_busy[psg_l_busy['Month'] == 'TOTAL'].loc[:, psg_l_busy.columns != 'Month']

combine_psg_airfare_less_busy = airfare_l_busy_yearly.merge(p_l_b_total_rows, left_on=('Year','City'), right_on=('Year','City'))
combine_psg_airfare_less_busy

# airfare_l_busy_yearly

In [ ]:
columns_to_convert = ['DOMESTIC', 'INTERNATIONAL', 'TOTAL']

for column in columns_to_convert:
    combine_psg_airfare_less_busy[column] = combine_psg_airfare_less_busy[column].str.replace(',', '').astype(int)

combine_psg_airfare_less_busy['INTERNATIONAL'] = combine_psg_airfare_less_busy['INTERNATIONAL'].apply(lambda x : x / (10**4))
combine_psg_airfare_less_busy['DOMESTIC'] = combine_psg_airfare_less_busy['DOMESTIC'].apply(lambda x : x / (10**4))
combine_psg_airfare_less_busy['TOTAL'] = combine_psg_airfare_less_busy['TOTAL'].apply(lambda x : x / (10**4))
combine_psg_airfare_less_busy

We group the yearly value for each year across cities. This grouping helps with combining the passenger count and the airfare data.

In [ ]:
grouped_airfare_less_busy = airfare_l_busy.groupby(by=['Year','City']).mean().reset_index()
grouped_airfare_less_busy.drop(columns='Quarter', inplace=True)
grouped_airfare_less_busy

### International flight Plot across City for less Busy Airports

In [ ]:
less_busy_cities = ['Boise','Kansas City','Oakland','Sacramento','Santa Ana']
l_busy_airfares = {city : grouped_airfare_less_busy.City == city for city in less_busy_cities}

In [ ]:
international_less_busy_cities = []

for city,mask in l_busy_airfares.items():
     international_less_busy_cities.append(
        combine_psg_airfare_less_busy[combine_psg_airfare_less_busy.City == city].hvplot.line(y='Current_City_Avg',x='INTERNATIONAL',label = f'{city}')
        * combine_psg_airfare_less_busy[combine_psg_airfare_less_busy.City == city].hvplot.scatter(
             y='Current_City_Avg',x='INTERNATIONAL', label = f'{city}').opts(tools = ['hover'])
        )

(
     international_less_busy_cities[0] * international_less_busy_cities[1] * international_less_busy_cities[2] * international_less_busy_cities[3] 
   * international_less_busy_cities[4]
   
).opts(
    title = 'International Passengers count in less busy airports and airfare across Cities',
    xlabel = 'International Passengers ($10^{4}$)',
    ylabel = 'Current_City_Avg Airfare value in USD',
    height=700,
    width = 1300,
    show_grid=True,
    xlim = (-50, 25000)
)

From the above plot, we are able to see that `Sacramento` has a huge difference in the number of International passengers along with the airfare

### Domestic data for Less Busy airports across city


In [ ]:
domestic_less_busy_cities = []

for city,mask in l_busy_airfares.items():
     domestic_less_busy_cities.append(
        combine_psg_airfare_less_busy[combine_psg_airfare_less_busy.City == city].hvplot.line(y='Current_City_Avg',x='DOMESTIC',label = f'{city}')
        * combine_psg_airfare_less_busy[combine_psg_airfare_less_busy.City == city].hvplot.scatter(y='Current_City_Avg',x='DOMESTIC', label = f'{city}').opts(tools = ['hover'])
        )

(
     domestic_less_busy_cities[0] * domestic_less_busy_cities[1] * domestic_less_busy_cities[2] * domestic_less_busy_cities[3] 
   * domestic_less_busy_cities[4]
   
).opts(
    title = 'Domestic Passengers count in less busy airports and airfare across Cities',
    xlabel = 'Domestic Passengers ($10^{4}$)',
    ylabel = 'Current_City_Avg Airfare value in USD',
    height=700,
    width = 1300,
    show_grid=True,
    xlim = (-50, 90000)
)

### Domestic Count across busy and less busy airports

In [ ]:
(
  domestic_busy_cities[0] * domestic_busy_cities[1] * domestic_busy_cities[2] * domestic_busy_cities[3] 
   * domestic_busy_cities[4] *
  domestic_less_busy_cities[0] * domestic_less_busy_cities[1] * domestic_less_busy_cities[2] * domestic_less_busy_cities[3] 
   * domestic_less_busy_cities[4]
   
 ).opts(
    title = 'Domestic Passengers count in airports and airfare across Cities',
    xlabel = 'Domestic Passengers ($10^{4}$)',
    ylabel = 'Current_City_Avg Airfare value in USD',
    height=700,
    width = 1300,
    show_grid=True,
    xlim = (0, 5000),
    ylim = (0, 500)
)

### International Count across busy and less busy airports

In [ ]:

(
   international_busy_cities[0] * international_busy_cities[1] * international_busy_cities[2] * international_busy_cities[3] * international_busy_cities[4]
   * international_less_busy_cities[0] * international_less_busy_cities[1] * international_less_busy_cities[2] * international_less_busy_cities[3] 
   * international_less_busy_cities[4]
).opts(
    title = 'International Passengers  airfare across cities',
    xlabel = 'International Passengers ($10^{4}$)',
    ylabel = 'Current_City_Avg Airfare value in USD',
    tools = ['hover'],
    height=700,
    width = 1200,
    show_grid=True,
    xlim = (-50, 700),
    ylim = (0, 800)
)

In [ ]:
domestic_less_busy_cities = []

for city,mask in l_busy_airfares.items():
     domestic_less_busy_cities.append(
        combine_psg_airfare_less_busy[combine_psg_airfare_less_busy.City == city].hvplot.line(y='Current_City_Avg',x='DOMESTIC',label = f'{city}')
        * combine_psg_airfare_less_busy[combine_psg_airfare_less_busy.City == city].hvplot.scatter(y='Current_City_Avg',x='DOMESTIC', label = f'{city}').opts(tools = ['hover'])
        )

(
     domestic_less_busy_cities[0] * domestic_less_busy_cities[1] * domestic_less_busy_cities[2] * domestic_less_busy_cities[3] 
   * domestic_less_busy_cities[4]
   
).opts(
    title = 'Domestic Passengers count in less busy airports and airfare across Cities',
    xlabel = 'Domestic Passengers ($10^{4}$)',
    ylabel = 'Current_City_Avg Airfare value in USD',
    height=700,
    width = 1300,
    show_grid=True,
    xlim = (-50, 800)
)

domestic_busy_cities = []

for city,mask in airfares.items():
    domestic_busy_cities.append(
    combine_psg_airfare[combine_psg_airfare.City == city].hvplot.line(y='Current_City_Avg',x='DOMESTIC',label = f'{city}')
    * combine_psg_airfare[combine_psg_airfare.City == city].hvplot.scatter(y='Current_City_Avg',x='DOMESTIC', label = f'{city}')
    )

(
    domestic_busy_cities[0] *
   domestic_busy_cities[1] * domestic_busy_cities[2] * domestic_busy_cities[3] * domestic_busy_cities[4]
).opts(
    title = 'Domestic Passengers in busy airports and airfare from 2003 till 2023',
    xlabel = 'Domestic Passengers ($10^{4}$)',
    ylabel = 'Current_City_Avg Airfare value in USD',
    height=700,
    width = 1000,
    show_grid=True,
    xlim = (400, 5000)
)

## Comparing International data across 10 cities

In [ ]:
combine_psg_airfare_less_busy.drop(columns='Quarter', inplace=True)

In [ ]:
domestic_data = pd.concat([combine_psg_airfare,combine_psg_airfare_less_busy])

- Here for this dynamic plot, we use `buttons` to indicate the `cities`.
- Since the passengers and the airfare values are numbers, we use the `Range Slider`.
- The `Year` being a list is also used to provide dynamic input to teh scatter plot for comparing the Passengers across cities.

In [ ]:

busy_cities = ['Atlanta', 'Dallas', 'New York', 'Seattle', 'Philadelphia']
less_busy_cities = ['Boise','Kansas City','Oakland','Sacramento','Santa Ana']

button_group = pn.widgets.CheckButtonGroup(
            name='success', button_type='success', button_style='outline'
            , options=['Atlanta', 'Dallas', 'New York', 'Seattle', 'Philadelphia','Boise','Kansas City','Oakland','Sacramento','Santa Ana']
            , value=['New York', 'Seattle', 'Sacramento']
        )

x_slider = pn.widgets.RangeSlider(
    name='Passenger Count',
    start=0,
    end=25000,
    step=25,
    value=(0, 25000)
)

y_slider = pn.widgets.RangeSlider(
    name='Avg City Airfare',
    start=200,
    end=1600,
    step=25,
    value=(0, 1600)
)

year_slider = pn.widgets.RangeSlider(
    name='Year',
    start=2003,
    end=2023,
    step=1,
    value=(2003, 2023)
)

# tell the plotting up front what it depends on!, adding a decorator(@)
@pn.depends(labels=button_group, x_range=x_slider, y_range=y_slider, year_mask=year_slider)
def plot_my_data(labels, x_range, y_range, year_mask):
    
    mask = ((domestic_data.City.isin(labels)) & domestic_data.INTERNATIONAL.between(*x_range) & domestic_data.Current_City_Avg.between(*y_range)
             & domestic_data.Year.between(*year_mask))
    return (domestic_data[mask].hvplot.scatter(
        y='Current_City_Avg',
        x='INTERNATIONAL',
        hover_cols = ['Year'],
        color = 'City',
        size = 40,
        yformatter='$ %.0f',
        # xformatter = '%.0f*10^4'
        xlabel = 'INTERNATIONAL($10^{4}$)'
    ) 
    ).opts(
        title='International Passenger Data across Cities',
        fontsize = 14
    )

# construct a `hv.DynamicMap` from out plotting function!
pn.Column(
    button_group,
   pn.Row(
        hv.DynamicMap(plot_my_data).opts(
        width = 900,
        height = 600
    ),
    pn.Column(    
        x_slider,
        y_slider,
        year_slider
    )
   )
) 


In [ ]:

busy_cities = ['Atlanta', 'Dallas', 'New York', 'Seattle', 'Philadelphia']
less_busy_cities = ['Boise','Kansas City','Oakland','Sacramento','Santa Ana']

button_group = pn.widgets.CheckButtonGroup(
            name='success', button_type='success', button_style='outline'
            , options=['Atlanta', 'Dallas', 'New York', 'Seattle', 'Philadelphia','Boise','Kansas City','Oakland','Sacramento','Santa Ana']
            , value=['New York', 'Seattle', 'Sacramento']
        )


x_slider = pn.widgets.RangeSlider(
    name='Passenger Count',
    start=0,
    end=80000,
    step=25,
    value=(0, 80000)
)

y_slider = pn.widgets.RangeSlider(
    name='Avg City Airfare',
    start=200,
    end=1600,
    step=25,
    value=(0, 1600)
)

year_slider = pn.widgets.RangeSlider(
    name='Year',
    start=2003,
    end=2023,
    step=1,
    value=(2003, 2023)
)

# tell the plotting up front what it depends on!, adding a decorator(@)
@pn.depends(labels=button_group, x_range=x_slider, y_range=y_slider, year_mask=year_slider)
def plot_my_data(labels, x_range, y_range, year_mask):
    
    mask = ((domestic_data.City.isin(labels)) & domestic_data.DOMESTIC.between(*x_range) 
            & domestic_data.Current_City_Avg.between(*y_range) & domestic_data.Year.between(*year_mask))
    return (domestic_data[mask].hvplot.scatter(
        y='Current_City_Avg',
        x='DOMESTIC',
        hover_cols = ['Year'],
        color = 'City',
        size = 40,
        yformatter='$ %.0f',
        # xformatter = '%.0f*10^4'
        xlabel = 'DOMESTIC (in $10^{4}$)'
    ) 
    ).opts(
        title='DOMESTIC Passenger Data across Cities',
        fontsize = 14
    )

# construct a `hv.DynamicMap` from out plotting function!
pn.Column(
    button_group,
   pn.Row(
        hv.DynamicMap(plot_my_data).opts(
        width = 900,
        height = 600
    ),
    pn.Column(    
        x_slider,
        y_slider,
        year_slider
    )
   )
) 


## GeoViews Plot on different cities

Here we are plotting the different cities on USA map wherein the color is based on `City` and the size is based on `Latitude`.

In [ ]:
dict_busy_cities = {'Boise': 'Boise, ID','Kansas City': 'Kansas City, MO','Oakland': 'Oakland, CA','Sacramento': 'Sacramento, CA',
                    'Santa Ana': 'Santa Ana, CA', 'Atlanta': 'Atlanta, GA', 'Dallas': 'Dallas, TX' , 'New York': 'New York City, NY', 'Seattle': 'Seattle, WA', 'Philadelphia': 'Philadelphia, PA'}           

geocities = []

for row in domestic_data['City']:
    if row in dict_busy_cities:
        geocities.append(dict_busy_cities[row])
    else:
        print('row', row)

domestic_data['GeoCity'] = geocities

The get_location function retrieves the latitude and longitude using the name of the city with the help of `geopandas`.

`Note:` The below cell takes nearly 2 minutes to run as it is retrieving the `Longitude` and `Latitude` data. 

In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="MyApp")

# define a function named get_location to get the lat/lon using geopy!
def get_location(row):
    location = geolocator.geocode(row.GeoCity)
    return location.latitude, location.longitude

domestic_data[['Latitude','Longitude']] = domestic_data.apply(get_location, axis=1, result_type = 'expand')
# we need to tell pandas that we are getting more than one column, here
# we need to expand the output into multiple column in our dataframe

geo_domestic_data = gpd.GeoDataFrame(
    domestic_data, geometry=gpd.points_from_xy(domestic_data.Longitude, domestic_data.Latitude)
)
geo_domestic_data[['Year', 'City','Current_City_Avg',	'Infl_Adj_City_Avg' ,'DOMESTIC', 'INTERNATIONAL', 'TOTAL', 'GeoCity', 'Latitude','Longitude', 'geometry']]

For this particular Geographical plot, we are considering the `Open Street Map`, which depicts the cities and the roadways

In [ ]:

world = gv.tile_sources.OSM().opts(
    height=480,
    width=640
)

In [ ]:
geo_domestic_data.info()

The hover tools helps to customize the values displayed when cursor hovers over the specific points.
We have the `City Name` as `index`, `Longitude`, `Latitude` displayed corresponding to `Year` as description

In [ ]:

hover = HoverTool(
    tooltips=[
            ("City", "@City"),
            ("Current_City_Avg", "$ @Current_City_Avg"), 
            ("Infl_Adj_City_Avg", "$ @Infl_Adj_City_Avg"),
            ("Current_US_Avg", "$ @Current_US_Avg"), 
            ("Infl_Adj_US_Avg", "$ @Infl_Adj_US_Avg")

        ]
    )

legend = Legend(location = (0, -30))

busy_geo = world * gv.Points(geo_domestic_data.iloc[100:105]).opts(
    tools = [hover],
    show_legend= True,
    size = gv.dim('Current_City_Avg')/25,
    color = 'Infl_Adj_City_Avg',
    cmap='viridis',
    height = 700,
    width = 1000,
    title = 'Inflation effect on Flight Prices [Busy Airports] 2022'
)

l_busy_geo = world * gv.Points(geo_domestic_data.iloc[-10:-5]).opts(
    tools = [hover],
    show_legend= True,
    size = gv.dim('Current_City_Avg')/25,
    color = 'Infl_Adj_City_Avg',
    cmap='viridis',
    height = 700,
    width = 1000,
    title = 'Inflation effect on Flight Prices [Less Busy Airports] 2022'
)

pn.Row(
    busy_geo,
    l_busy_geo
)

## Tabulator widget

The `domestic_data` dataframe contains the complete international, domestic data across busy and less busy cities.

In [ ]:
domestic_data.info()

In [ ]:
from bokeh.models.widgets.tables import NumberFormatter, DateFormatter

bokeh_formatters = {
    'float64': NumberFormatter(format='0.00'),
    'int32': NumberFormatter(format='0000'),
    'date': DateFormatter(format='%Y '),
}

In [ ]:
pn.extension('tabulator')
domestic_data_tab = domestic_data[['Year', 'GeoCity', 'Current_City_Avg', 'Infl_Adj_City_Avg', 'DOMESTIC', 'INTERNATIONAL', 'TOTAL']]

domestic_data_tb = pn.widgets.Tabulator(
    domestic_data_tab,
    page_size=10,
    pagination='local',
    selectable=True,
    disabled=False
)

# provide a slider to filter out heroes with low pick counts
pick_slider = pn.widgets.IntSlider(
    name='Minimum Air Fare',
    start=100,
    end=int(domestic_data.Current_City_Avg.max()),
    step=50
)

@pn.depends(pick_slider, watch=True)
def filtered_hero_picks(value):
    domestic_data_tb._filters = []
    domestic_data_tb.add_filter((value, pick_slider.end), 'Current_City_Avg')

pn.Column(pick_slider, domestic_data_tb)


### Correlation coefficient

The correlation coefficient coefficient between each of the columns in data is provided below.

#### INTERNATIONAL Passengers vs Current City Average on Airfare

In [ ]:
np.corrcoef(domestic_data.INTERNATIONAL, domestic_data.Current_City_Avg)

#### DOMESTIC Passengers vs Inflation adjusted City Average on Airfare

In [ ]:
np.corrcoef(domestic_data.DOMESTIC, domestic_data.Infl_Adj_City_Avg)

#### DOMESTIC Passengers vs Current City Average on Airfare

In [ ]:
np.corrcoef(domestic_data.DOMESTIC, domestic_data.Current_City_Avg)

#### INTERNATIONAL Passengers vs Inflation Adjusted City Average on Airfare

In [ ]:
np.corrcoef(domestic_data.INTERNATIONAL, domestic_data.Infl_Adj_City_Avg)

#### DOMESTIC Passengers vs Inflation Adjusted City Average on Airfare

In [ ]:
np.corrcoef(domestic_data.TOTAL, domestic_data.Infl_Adj_City_Avg)

#### DOMESTIC Passengers vs Current City Average on Airfare

In [ ]:
np.corrcoef(domestic_data.TOTAL, domestic_data.Current_City_Avg)

### Conclusion